# 웹 크롤링

## 1. selenium

동적으로 생성된 웹 콘텐츠 (JavaScript)를 처리하고 스크래핑 할 때 유리

## 2. BeautifulSoup

정적으로 생성된 웹 콘텐츠 (HTML)을 처리하고 스크래핑 할 때 유리

-  두가지를 적절하게 사용해 퍼포먼스 강화

## 멜론 top 100 월간 차트

In [11]:
""" 
xpath 경로 복사!
XPath는 XML Path Language의 약자로, XML 및 HTML 문서 내에서 특정 요소나 속성을 선택하기 위한 언어입니다
"""

# 멜론차트 : //*[@id="gnb_menu"]/ul[1]/li[1]/a/span[2]
# 월간    : //*[@id="gnb_menu"]/ul[1]/li[1]/div/ul/li[5]/a/span
# 월 선택  : //*[@id="conts"]/div[3]/div/button
# 1월     : //*[@id="conts"]/div[3]/div/div/dl/dd[1]/ul/li[1]/a
# 2월     : //*[@id="conts"]/div[3]/div/div/dl/dd[1]/ul/li[2]/a/span

' \nxpath 경로 복사!\nXPath는 XML Path Language의 약자로, XML 및 HTML 문서 내에서 특정 요소나 속성을 선택하기 위한 언어입니다\n'

In [40]:
'''
%pip install webdriver-manager
%pip install selenium
!pip install beautifulsoup4
'''
#%pip install --upgrade webdriver-manager

SyntaxError: invalid syntax (156006033.py, line 7)

In [46]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 크롬 옵션 지정
options = webdriver.ChromeOptions()

# 크롤링 막는 것을 피하기 위해 에이전트 입력 (사람처럼 보이게 하기)
UserAgent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
options.add_argument('user-agent=' + UserAgent)

# 드라이버 설정 (자동 다운로드 및 적용)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(10)
driver.get(url='https://www.melon.com/')

# 멜론 차트 이동
driver.find_element(By.XPATH, '//*[@id="gnb_menu"]/ul[1]/li[1]/a/span[2]').click()
time.sleep(1)

# 월간 차트 이동
driver.find_element(By.XPATH, '//*[@id="gnb_menu"]/ul[1]/li[1]/div/ul/li[5]/a/span').click()
time.sleep(1)

# 월 선택
driver.find_element(By.XPATH, '//*[@id="conts"]/div[3]/div/button').click()
time.sleep(1)
driver.find_element(By.XPATH, '//*[@id="conts"]/div[3]/div/div/dl/dd[1]/ul/li[12]/a').click()
time.sleep(1)

# 곡 정보 추출
song_data = []


for rank in range(1, 101):  # 상위 5곡만 추출 (테스트 목적)
    #for rank in range(1, 101):  # 상위 100곡 추출
    # 곡 리스트 가져오기
    if rank <= 50:
        xpath = '//tr[contains(@class, "lst50")]'
    else:
        xpath = '//tr[contains(@class, "lst100")]'

    songs = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, xpath))
    )
    song = songs[(rank - 1) % 50]  # 1~50과 51~100의 인덱스를 조정

    # 곡 고유 ID 추출
    song_id = song.get_attribute("data-song-no")

    # 제목과 가수 정보 추출
    title_element = song.find_element(By.XPATH, './td[6]/div/div/div[1]/span/a')
    artist_element = song.find_element(By.XPATH, './td[6]/div/div/div[2]/a')

    song_title = title_element.text
    song_artist = artist_element.text

    # 곡 상세 페이지 이동 (고유 ID 사용)
    driver.get(f'https://www.melon.com/song/detail.htm?songId={song_id}')
    time.sleep(0.5)

    # 장르 정보 추출
    try:
        soup = bs(driver.page_source, 'html.parser')
        genre = soup.select_one('dl.list > dd:nth-of-type(3)').text.strip()
    except AttributeError:
        genre = '정보 없음'

    # 데이터 저장
    song_data.append({
        '순위': rank,
        '제목': song_title,
        '가수': song_artist,
        '장르': genre
    })

    # 월간 차트 1월 URL로 돌아가기
    driver.get('https://www.melon.com/chart/month/index.htm')
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="conts"]/div[3]/div/button').click()
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="conts"]/div[3]/div/div/dl/dd[1]/ul/li[12]/a').click()
    time.sleep(0.5)

# CSV 파일로 저장
df = pd.DataFrame(song_data)
print(df)

driver.quit()

     순위                              제목         가수    장르
0     1  HOME SWEET HOME (feat. 태양, 대성)   G-DRAGON  랩/힙합
1     2                            APT.  로제 (ROSÉ)    댄스
2     3                        Whiplash      aespa    댄스
3     4                          나는 반딧불        황가람   발라드
4     5                           POWER   G-DRAGON  랩/힙합
..  ...                             ...        ...   ...
95   96                     가까운 듯 먼 그대여       카더가든   발라드
96   97                       Do or Die        임영웅    댄스
97   98                         버추얼 아이돌      PLAVE  랩/힙합
98   99                      London Boy        임영웅  록/메탈
99  100                          오래된 노래         허각   발라드

[100 rows x 4 columns]


In [ ]:
df.to_csv('melon_chart_12month.csv', index=False, encoding='utf-8-sig')